# PyTorch Tensors

In [1]:
import torch

In [2]:
a = torch.ones(3)

In [3]:
a

tensor([1., 1., 1.])

In [8]:
# shape
a.shape

torch.Size([3])

In [9]:
# can traverse
a[1]

tensor(1.)

In [10]:
# can slice
a[:2]

tensor([1., 1.])

In [13]:
# mutable
a[2] = 3.

In [14]:
a

tensor([1., 1., 3.])

**NOTE:** tensors are stored as contiguous memory blocks as float32 (32bits/4bytes) per element

In [22]:
# createing tensors from list
b = torch.tensor([1.2, 2.3, 3.4, 4.5])
b, b.shape

(tensor([1.2000, 2.3000, 3.4000, 4.5000]), torch.Size([4]))

In [23]:
b = torch.tensor([[1.2, 2.3],[3.4, 4.5]])
b, b.shape

(tensor([[1.2000, 2.3000],
         [3.4000, 4.5000]]),
 torch.Size([2, 2]))

In [27]:
# accessing entry at index 1
b[1:]

tensor([[3.4000, 4.5000]])

In [26]:
b[:,0]

tensor([1.2000, 3.4000])

### Giving names to dimensions
we can add names to the dimensions using the function refine names

In [37]:
b_named = b.refine_names(...,'rows','columns')

/Users/user/AuDev/pytorch/lib/python3.9/site-packages/torch/_tensor.py:780: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1156.)
  return super(Tensor, self).refine_names(names)


In [38]:
print(b_named.shape, b_named.names)

torch.Size([2, 2]) ('rows', 'columns')


Functions that take dimension arguments also accept named dimensions

In [39]:
summed_col = b_named.sum('columns')
summed_row = b_named.sum('rows')

In [42]:
print(b_named)
print("Summed on Columns: ",summed_col)
print("Summed on Rows: ",summed_row)

tensor([[1.2000, 2.3000],
        [3.4000, 4.5000]], names=('rows', 'columns'))
Summed on Columns:  tensor([3.5000, 7.9000], names=('rows',))
Summed on Rows:  tensor([4.6000, 6.8000], names=('columns',))


**NOTE:** Once named, we cannot combine dimensions with different names i.e. standard broadcasting would not apply

to use functions that do not accept named dimensions we will need to remove the names

In [43]:
b_named = b_named.rename(None)

In [44]:
b_named

tensor([[1.2000, 2.3000],
        [3.4000, 4.5000]])

In [45]:
b_named.names

(None, None)

### DataTypes of tensors
- torch.float32 or torch.float: 32-bit floating-point
- torch.float64 or torch.double: 64-bit, double-precision floating-point 
- torch.float16 or torch.half: 16-bit, half-precision floating-point
- torch.int8: signed 8-bit integers
- torch.uint8: unsigned 8-bit integers
- torch.int16 or torch.short: signed 16-bit integers
- torch.int32 or torch.int: signed 32-bit integers
- torch.int64 or torch.long: signed 64-bit integers
- torch.bool: Boolean

In [46]:
# We can explicitly mention the dtype for a tensor
double_points = torch.ones(10,2,dtype=torch.double)

In [47]:
double_points.dtype

torch.float64

In [49]:
# another way
double_points = torch.zeros(10,2).to(torch.short)

In [50]:
double_points.dtype

torch.int16

### Storage of Tensors
Values in tensors are allocated in contiguous chunks of memory managed by torch.Storage instances. A storage is a one-dimensional array of numerical data: that is, a contiguous block of memory containing numbers of a given type, such as float (32 bits repre- senting a floating-point number) or int64 (64 bits representing an integer). A PyTorch Tensor instance is a view of such a Storage instance that is capable of indexing into that storage using an offset and per-dimension strides.

In [54]:
b.storage()

 1.2000000476837158
 2.299999952316284
 3.4000000953674316
 4.5
[torch.FloatStorage of size 4]

In [56]:
b.storage()[2]

3.4000000953674316

The indexing of storage is always 1D

while storage the tensors need the following:
- Size: The actual shape of the tensord eg: (3,3) for a 3x3 tensor
- Offset: The storage offset is the index in the storage corresponding to the first element in the tensor
- Stride: The stride is the number of elements in the storage that need to be skipped over to obtain the next element along each dimension

<img src="assets/pytorch101_3.5.png" width='750'>


In [60]:
b[1].storage_offset()

2

In [62]:
b.stride()

(2, 1)

**Accessing an element i, j in a 2D tensor results in accessing the `storage_offset + stride[0] * i + stride[1] * j` element in the storage**

A transpose would basically access the same storage with different offset and stride (new tensor object with same memory storage but different size, storage offset, or stride)